In [1]:
from loader.dataloader import MIDI_Loader,MIDI_Render
import numpy as np
import os
import shutil
import random
from model_mask_cond import MaskNN
from vae.model import VAE
import torch
import sklearn.utils
from torch.nn import functional as F

vector_num = 256
cond_num = 12
rest_pitch = 129
hold_pitch = 128

train_path = "processed_data/vae_train_data.npy"
validate_path = "processed_data/vae_validate_data.npy"
test_path = "processed_data/vae_test_data.npy"

total_len = 256 # 32.0 s
known_len = 128 # 16.0 s
shift_len = 32 # 4.0 s
total_epoch = 2000
batch_size = 64
vae_len = 32
learning_rate =1e-4


def make_one_hot_data(train_data):
    print("convert data to one-hot...")
    train_size = min(len(train_data),15000)
    train_x = []
    train_gd = []

    # process with bi-directional issue

    for i,data in enumerate(train_data):
        if i >= train_size:
            break
        mi = data["notes"]
        ci = data["chords"]
        train_xx = np.zeros((len(mi),pitch_num + chord_num), dtype = np.int32)
        train_gdd = np.zeros((len(mi)), dtype = np.int32)
        prev = rest_pitch
        for j, value in enumerate(mi):
            if j < known_len:
                if value != hold_pitch:
                    prev = value
                if value == hold_pitch and mi[j + 1] != hold_pitch:
                    train_xx[j,prev] = 1
                elif j + 1 == known_len and value == hold_pitch:
                    train_xx[j,prev] = 1
                else:
                    train_xx[j,value] = 1
        for j, value in enumerate(ci):
            train_xx[j, value + pitch_num] = 1
        prev = rest_pitch
        for j, value in enumerate(mi):
            if value != hold_pitch:
                prev = value
            if j + 1 == len(mi):
                train_gdd[j] = prev
            elif value == hold_pitch and mi[j + 1] != hold_pitch:
                train_gdd[j] = prev
            else:
                train_gdd[j] = value
        train_x.append(train_xx)
        train_gd.append(train_gdd)
    print("convert success！")
    train_x = np.array(train_x)
    train_gd = np.array(train_gd)
    return [train_x,train_gd]


test_data = np.load("vae_test_data_no_split.npy",allow_pickle = True)
test_z1 = test_data[:,0]
test_z2 = test_data[:,1]
test_chord = test_data[:,2]

print(test_data.shape)
# test_x,test_gd = make_one_hot_data(test_data)
print(test_z1.shape)
print(test_z2.shape)
print(test_chord.shape)

print(test_z1[421].shape)
print(test_z2[421].shape)
print(test_chord[421].shape)
print(test_chord[421][5])

(2064, 3)
(2064,)
(2064,)
(2064,)
torch.Size([8, 128])
(8, 128)
(8, 32, 12)
[[0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0

In [2]:
model = MaskNN(input_dims = vector_num ,
        hidden_dims = vector_num + 128,output_dims = 128,time_steps = total_len)

dic = torch.load("model_v5.pt")
for name in list(dic.keys()):
    dic[name.replace('module.', '')] = dic.pop(name)
model.load_state_dict(dic)
if torch.cuda.is_available():
    print("Using:", torch.cuda.get_device_name(torch.cuda.current_device()),flush = True)
    model.cuda()
else:
    print("Using CPU",flush = True)



Using: GeForce GTX 1080 Ti


In [3]:
print(model)

MaskNN(
  (LTSM_e1): LSTM(256, 384, batch_first=True, bidirectional=True)
  (dropout_1): Dropout(p=0.5, inplace=False)
  (LTSM_e2): LSTM(768, 384, batch_first=True, bidirectional=True)
  (dropout_2): Dropout(p=0.5, inplace=False)
  (LTSM_e3): LSTM(768, 384, batch_first=True, bidirectional=True)
  (dropout_3): Dropout(p=0.5, inplace=False)
  (LTSM_e4): LSTM(768, 384, batch_first=True, bidirectional=True)
  (LTSM_d1): LSTM(768, 384, batch_first=True, bidirectional=True)
  (dropout_4): Dropout(p=0.5, inplace=False)
  (LTSM_d2): LSTM(768, 384, batch_first=True, bidirectional=True)
  (dropout_5): Dropout(p=0.5, inplace=False)
  (LTSM_d3): LSTM(768, 384, batch_first=True, bidirectional=True)
  (dropout_6): Dropout(p=0.5, inplace=False)
  (LTSM_d4): LSTM(768, 384, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=768, out_features=128, bias=True)
)


In [5]:
total_len = 8 # 32.0 s
known_len = 4 # 16.0 s
shift_len = 1 # 4.0 s
new_outputs = []
test_data_real = np.load("test_data_no_split_8_crop.npy",allow_pickle=True)
for i in range(len(test_data)):
    print(i)
    z1s = test_z1[i]
    z2s = test_z2[i]
    chords = test_chord[i]
    new_output = np.concatenate((z1s,z2s), 1)
    final_z = new_output[-1]
    print(new_output.shape)
    for dpos in range(0,len(new_output) - total_len + 1, known_len):
        temp_input = new_output[dpos:dpos + total_len]
#         print(dpos)
        temp_input[known_len:,:128] = 0
#         print(temp_input.shape)
#         print(temp_input)
        temp_input = np.expand_dims(temp_input, axis = 0)
        temp_input = torch.from_numpy(temp_input).float()

#         print(torch.argmax(temp_input[:,:,:pitch_num],dim = 2))
        if torch.cuda.is_available():
            temp_input = temp_input.cuda()
        model_out = model(temp_input).detach()
        model_out = torch.squeeze(model_out)
        model_out = model_out[known_len:total_len]
        model_out = model_out.cpu().numpy()
        print(model_out.shape)
#         print(model_out)
        for j in range(len(model_out)):
            new_output[dpos + j + known_len,:128] = model_out[j]
#         print(new_output)
#         print(new_output.shape)
    new_output[-1] = final_z
    new_outputs.append([new_output,chords])
#     break
# print(new_outputs[0])
#     new_outputs.append({"notes":np.argmax(new_output[:,:pitch_num],axis = 1),"chords": np.argmax(new_output[:,pitch_num:pitch_num+chord_num],axis = 1)})
print(len(new_outputs))
# vae_model = VAE(130, 2048, 3, 12, 128, 128, 32)
# vae_model.eval()
# dic = torch.load("vae/tr_chord.pt")
# for name in list(dic.keys()):
#     dic[name.replace('module.', '')] = dic.pop(name)
# vae_model.load_state_dict(dic)
# if torch.cuda.is_available():
#     vae_model = vae_model.cuda()
    
# z1 = new_outputs[0][0][:,:128]
# z2 = new_outputs[0][0][:,128:]
# chord_cond = new_outputs[0][1]
# print(z1.shape)
# print(z2.shape)
# print(chord_cond.shape)
# z1 = torch.from_numpy(z1).float()
# z2 = torch.from_numpy(z2).float()
# chord_cond = torch.from_numpy(chord_cond).float()
# z1 = z1.cuda()
# z2 = z2.cuda()
# chord_cond = chord_cond.cuda()
# res = vae_model.decoder(z1,z2,chord_cond)
# print(res.detach().cpu().numpy().shape)
# q = np.asarray((res.detach().cpu().numpy()))
# temp = []
# for i in q:
#     for j in i:
#         temp.append(np.argmax(j))
# q = np.array(temp)

# song = {"notes":q, "chords": test_data_real[0]["chords"]}
# render = MIDI_Render(datasetName = "Nottingham", minStep= 0.125)
# render.data2midi(data = song,output = "test.mid")
# render.data2midi(data = test_data_real[0],output = "real.mid")

#         print(torch.argmax(temp_input[:,:,:pitch_num],dim = 2))
#         break
#     break

0
(12, 256)
(4, 128)
(4, 128)
1
(16, 256)
(4, 128)
(4, 128)
(4, 128)
2
(15, 256)
(4, 128)
(4, 128)
3
(18, 256)
(4, 128)
(4, 128)
(4, 128)
4
(6, 256)
5
(16, 256)
(4, 128)
(4, 128)
(4, 128)
6
(12, 256)
(4, 128)
(4, 128)
7
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
8
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
9
(4, 256)
10
(8, 256)
(4, 128)
11
(4, 256)
12
(8, 256)
(4, 128)
13
(8, 256)
(4, 128)
14
(8, 256)
(4, 128)
15
(8, 256)
(4, 128)
16
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
17
(8, 256)
(4, 128)
18
(8, 256)
(4, 128)
19
(8, 256)
(4, 128)
20
(4, 256)
21
(8, 256)
(4, 128)
22
(12, 256)
(4, 128)
(4, 128)
23
(13, 256)
(4, 128)
(4, 128)
24
(12, 256)
(4, 128)
(4, 128)
25
(18, 256)
(4, 128)
(4, 128)
(4, 128)
26
(12, 256)
(4, 128)
(4, 128)
27
(12, 256)
(4, 128)
(4, 128)
28
(12, 256)
(4, 128)
(4, 128)
29
(12, 256)
(4, 128)
(4, 128)
30
(12, 256)
(4, 128)
(4, 128)
31
(12, 256)
(4, 128)
(4, 128)
32
(12, 256)
(4, 128)
(4, 128)
33
(12, 256)
(4, 128)
(4, 128)
34
(1

(4, 128)
(4, 128)
(4, 128)
(4, 128)
213
(4, 256)
214
(8, 256)
(4, 128)
215
(4, 256)
216
(8, 256)
(4, 128)
217
(8, 256)
(4, 128)
218
(8, 256)
(4, 128)
219
(8, 256)
(4, 128)
220
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
221
(8, 256)
(4, 128)
222
(8, 256)
(4, 128)
223
(8, 256)
(4, 128)
224
(4, 256)
225
(8, 256)
(4, 128)
226
(12, 256)
(4, 128)
(4, 128)
227
(13, 256)
(4, 128)
(4, 128)
228
(12, 256)
(4, 128)
(4, 128)
229
(18, 256)
(4, 128)
(4, 128)
(4, 128)
230
(12, 256)
(4, 128)
(4, 128)
231
(12, 256)
(4, 128)
(4, 128)
232
(12, 256)
(4, 128)
(4, 128)
233
(12, 256)
(4, 128)
(4, 128)
234
(12, 256)
(4, 128)
(4, 128)
235
(12, 256)
(4, 128)
(4, 128)
236
(12, 256)
(4, 128)
(4, 128)
237
(12, 256)
(4, 128)
(4, 128)
238
(12, 256)
(4, 128)
(4, 128)
239
(12, 256)
(4, 128)
(4, 128)
240
(12, 256)
(4, 128)
(4, 128)
241
(18, 256)
(4, 128)
(4, 128)
(4, 128)
242
(12, 256)
(4, 128)
(4, 128)
243
(9, 256)
(4, 128)
244
(12, 256)
(4, 128)
(4, 128)
245
(12, 256)
(4, 128)
(4, 128)
246
(12, 256)
(4, 12

(4, 128)
431
(13, 256)
(4, 128)
(4, 128)
432
(12, 256)
(4, 128)
(4, 128)
433
(18, 256)
(4, 128)
(4, 128)
(4, 128)
434
(12, 256)
(4, 128)
(4, 128)
435
(12, 256)
(4, 128)
(4, 128)
436
(12, 256)
(4, 128)
(4, 128)
437
(12, 256)
(4, 128)
(4, 128)
438
(12, 256)
(4, 128)
(4, 128)
439
(12, 256)
(4, 128)
(4, 128)
440
(12, 256)
(4, 128)
(4, 128)
441
(12, 256)
(4, 128)
(4, 128)
442
(12, 256)
(4, 128)
(4, 128)
443
(12, 256)
(4, 128)
(4, 128)
444
(12, 256)
(4, 128)
(4, 128)
445
(18, 256)
(4, 128)
(4, 128)
(4, 128)
446
(12, 256)
(4, 128)
(4, 128)
447
(9, 256)
(4, 128)
448
(12, 256)
(4, 128)
(4, 128)
449
(12, 256)
(4, 128)
(4, 128)
450
(12, 256)
(4, 128)
(4, 128)
451
(18, 256)
(4, 128)
(4, 128)
(4, 128)
452
(12, 256)
(4, 128)
(4, 128)
453
(18, 256)
(4, 128)
(4, 128)
(4, 128)
454
(12, 256)
(4, 128)
(4, 128)
455
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
456
(12, 256)
(4, 128)
(4, 128)
457
(18, 256)
(4, 128)
(4, 128)
(4, 128)
458
(12, 256)
(4, 128)
(4, 128)
459
(18, 256)
(4, 128)
(4, 128)
(

(4, 128)
(4, 128)
(4, 128)
638
(12, 256)
(4, 128)
(4, 128)
639
(12, 256)
(4, 128)
(4, 128)
640
(12, 256)
(4, 128)
(4, 128)
641
(12, 256)
(4, 128)
(4, 128)
642
(12, 256)
(4, 128)
(4, 128)
643
(12, 256)
(4, 128)
(4, 128)
644
(12, 256)
(4, 128)
(4, 128)
645
(12, 256)
(4, 128)
(4, 128)
646
(12, 256)
(4, 128)
(4, 128)
647
(12, 256)
(4, 128)
(4, 128)
648
(12, 256)
(4, 128)
(4, 128)
649
(18, 256)
(4, 128)
(4, 128)
(4, 128)
650
(12, 256)
(4, 128)
(4, 128)
651
(9, 256)
(4, 128)
652
(12, 256)
(4, 128)
(4, 128)
653
(12, 256)
(4, 128)
(4, 128)
654
(12, 256)
(4, 128)
(4, 128)
655
(18, 256)
(4, 128)
(4, 128)
(4, 128)
656
(12, 256)
(4, 128)
(4, 128)
657
(18, 256)
(4, 128)
(4, 128)
(4, 128)
658
(12, 256)
(4, 128)
(4, 128)
659
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
660
(12, 256)
(4, 128)
(4, 128)
661
(18, 256)
(4, 128)
(4, 128)
(4, 128)
662
(12, 256)
(4, 128)
(4, 128)
663
(18, 256)
(4, 128)
(4, 128)
(4, 128)
664
(12, 256)
(4, 128)
(4, 128)
665
(36, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 12

(4, 128)
(4, 128)
847
(12, 256)
(4, 128)
(4, 128)
848
(12, 256)
(4, 128)
(4, 128)
849
(12, 256)
(4, 128)
(4, 128)
850
(12, 256)
(4, 128)
(4, 128)
851
(12, 256)
(4, 128)
(4, 128)
852
(12, 256)
(4, 128)
(4, 128)
853
(18, 256)
(4, 128)
(4, 128)
(4, 128)
854
(12, 256)
(4, 128)
(4, 128)
855
(9, 256)
(4, 128)
856
(12, 256)
(4, 128)
(4, 128)
857
(12, 256)
(4, 128)
(4, 128)
858
(12, 256)
(4, 128)
(4, 128)
859
(18, 256)
(4, 128)
(4, 128)
(4, 128)
860
(12, 256)
(4, 128)
(4, 128)
861
(18, 256)
(4, 128)
(4, 128)
(4, 128)
862
(12, 256)
(4, 128)
(4, 128)
863
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
864
(12, 256)
(4, 128)
(4, 128)
865
(18, 256)
(4, 128)
(4, 128)
(4, 128)
866
(12, 256)
(4, 128)
(4, 128)
867
(18, 256)
(4, 128)
(4, 128)
(4, 128)
868
(12, 256)
(4, 128)
(4, 128)
869
(36, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
870
(12, 256)
(4, 128)
(4, 128)
871
(12, 256)
(4, 128)
(4, 128)
872
(12, 256)
(4, 128)
(4, 128)
873
(24, 256)
(4, 128)
(4, 128)
(4

(4, 128)
1056
(12, 256)
(4, 128)
(4, 128)
1057
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1058
(12, 256)
(4, 128)
(4, 128)
1059
(9, 256)
(4, 128)
1060
(12, 256)
(4, 128)
(4, 128)
1061
(12, 256)
(4, 128)
(4, 128)
1062
(12, 256)
(4, 128)
(4, 128)
1063
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1064
(12, 256)
(4, 128)
(4, 128)
1065
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1066
(12, 256)
(4, 128)
(4, 128)
1067
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1068
(12, 256)
(4, 128)
(4, 128)
1069
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1070
(12, 256)
(4, 128)
(4, 128)
1071
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1072
(12, 256)
(4, 128)
(4, 128)
1073
(36, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1074
(12, 256)
(4, 128)
(4, 128)
1075
(12, 256)
(4, 128)
(4, 128)
1076
(12, 256)
(4, 128)
(4, 128)
1077
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1078
(12, 256)
(4, 128)
(4, 128)
1079
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1080
(18, 256)
(4, 128)
(4, 1

(4, 128)
1257
(12, 256)
(4, 128)
(4, 128)
1258
(12, 256)
(4, 128)
(4, 128)
1259
(12, 256)
(4, 128)
(4, 128)
1260
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1261
(12, 256)
(4, 128)
(4, 128)
1262
(9, 256)
(4, 128)
1263
(12, 256)
(4, 128)
(4, 128)
1264
(12, 256)
(4, 128)
(4, 128)
1265
(12, 256)
(4, 128)
(4, 128)
1266
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1267
(12, 256)
(4, 128)
(4, 128)
1268
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1269
(12, 256)
(4, 128)
(4, 128)
1270
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1271
(12, 256)
(4, 128)
(4, 128)
1272
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1273
(12, 256)
(4, 128)
(4, 128)
1274
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1275
(12, 256)
(4, 128)
(4, 128)
1276
(36, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1277
(12, 256)
(4, 128)
(4, 128)
1278
(12, 256)
(4, 128)
(4, 128)
1279
(12, 256)
(4, 128)
(4, 128)
1280
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1281
(12, 256)
(4, 128)
(4, 128)
1282
(24, 256)
(4, 128)

(4, 128)
(4, 128)
1462
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1463
(12, 256)
(4, 128)
(4, 128)
1464
(9, 256)
(4, 128)
1465
(12, 256)
(4, 128)
(4, 128)
1466
(12, 256)
(4, 128)
(4, 128)
1467
(12, 256)
(4, 128)
(4, 128)
1468
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1469
(12, 256)
(4, 128)
(4, 128)
1470
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1471
(12, 256)
(4, 128)
(4, 128)
1472
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1473
(12, 256)
(4, 128)
(4, 128)
1474
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1475
(12, 256)
(4, 128)
(4, 128)
1476
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1477
(12, 256)
(4, 128)
(4, 128)
1478
(36, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1479
(12, 256)
(4, 128)
(4, 128)
1480
(12, 256)
(4, 128)
(4, 128)
1481
(12, 256)
(4, 128)
(4, 128)
1482
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1483
(12, 256)
(4, 128)
(4, 128)
1484
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1485
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1486
(12, 2

(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1666
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1667
(12, 256)
(4, 128)
(4, 128)
1668
(36, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1669
(12, 256)
(4, 128)
(4, 128)
1670
(12, 256)
(4, 128)
(4, 128)
1671
(12, 256)
(4, 128)
(4, 128)
1672
(12, 256)
(4, 128)
(4, 128)
1673
(12, 256)
(4, 128)
(4, 128)
1674
(12, 256)
(4, 128)
(4, 128)
1675
(12, 256)
(4, 128)
(4, 128)
1676
(12, 256)
(4, 128)
(4, 128)
1677
(12, 256)
(4, 128)
(4, 128)
1678
(12, 256)
(4, 128)
(4, 128)
1679
(12, 256)
(4, 128)
(4, 128)
1680
(18, 256)
(4, 128)
(4, 128)
(4, 128)
1681
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1682
(12, 256)
(4, 128)
(4, 128)
1683
(48, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1684
(47, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1685
(8, 256)
(4, 128)
1686
(34, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4

(4, 128)
(4, 128)
1866
(8, 256)
(4, 128)
1867
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1868
(60, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1869
(16, 256)
(4, 128)
(4, 128)
(4, 128)
1870
(8, 256)
(4, 128)
1871
(16, 256)
(4, 128)
(4, 128)
(4, 128)
1872
(16, 256)
(4, 128)
(4, 128)
(4, 128)
1873
(16, 256)
(4, 128)
(4, 128)
(4, 128)
1874
(16, 256)
(4, 128)
(4, 128)
(4, 128)
1875
(8, 256)
(4, 128)
1876
(16, 256)
(4, 128)
(4, 128)
(4, 128)
1877
(8, 256)
(4, 128)
1878
(16, 256)
(4, 128)
(4, 128)
(4, 128)
1879
(16, 256)
(4, 128)
(4, 128)
(4, 128)
1880
(16, 256)
(4, 128)
(4, 128)
(4, 128)
1881
(16, 256)
(4, 128)
(4, 128)
(4, 128)
1882
(16, 256)
(4, 128)
(4, 128)
(4, 128)
1883
(16, 256)
(4, 128)
(4, 128)
(4, 128)
1884
(24, 256)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
(4, 128)
1885
(16, 256)
(4, 128)
(4, 128)
(4, 128)
1886
(16, 256)
(4, 128)
(4, 128)
(4, 128)
1887
(16, 256)
(4, 128)
(4, 128)
(4, 128)

In [14]:
# vae_model = VAE(130, 2048, 3, 12, 128, 128, 32)
# vae_model.eval()
# dic = torch.load("vae/tr_chord.pt")
# for name in list(dic.keys()):
#     dic[name.replace('module.', '')] = dic.pop(name)
# vae_model.load_state_dict(dic)
# if torch.cuda.is_available():
#     vae_model = vae_model.cuda()
    
# z1 = new_outputs[0][0][:,:128]
# z2 = new_outputs[0][0][:,128:]
# chord_cond = new_outputs[0][1]
# print(z1.shape)
# print(z2.shape)
# print(chord_cond.shape)
# z1 = torch.from_numpy(z1).float()
# z2 = torch.from_numpy(z2).float()
# chord_cond = torch.from_numpy(chord_cond).float()
# z1 = z1.cuda()
# z2 = z2.cuda()
# chord_cond = chord_cond.cuda()
# res = vae_model.decoder(z1,z2,chord_cond)
# print(res.detach().cpu().numpy().shape)
# q = np.asarray((res.detach().cpu().numpy()))
# temp = []
# for i in q:
#     for j in i:
#         temp.append(np.argmax(j))
# q = np.array(temp)

# song = {"notes":q, "chords": test_data_real[0]["chords"]}
# render = MIDI_Render(datasetName = "Nottingham", minStep= 0.125)
# render.data2midi(data = song,output = "test.mid")
# render.data2midi(data = test_data_real[0],output = "real.mid")

(12, 128)
(12, 128)
(12, 32, 12)
(12, 32, 130)
finish render midi on test.mid
finish render midi on real.mid


In [6]:
np.save("model_v5_output.npy",new_outputs)

In [ ]:
print(len(new_outputs))